In [1]:
import os

import numpy as np
import json
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import (Input, Conv2D, ReLU, BatchNormalization, Add, GlobalAvgPool2D,
                          Dense, Multiply, ZeroPadding2D, MaxPooling2D, AveragePooling2D, Flatten)
from core.learning_tools import F1CombineMetric as F1_comb
from core.learning_tools import get_resnet50_model, f1_comb
from sklearn.metrics import confusion_matrix
from keras.callbacks import Callback
from keras.models import load_model
from keras.initializers import glorot_uniform
from core.learning_tools import ImageDataGenerator, send_telegram_msg

2023-09-21 15:35:53.355450: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
np.set_printoptions(suppress=True)

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
visible_devices = [physical_devices[2]]  # , physical_devices[1]
tf.config.set_visible_devices(visible_devices, 'GPU')

In [1]:
target_classes = ['standing', 'walking', 'no_class']


In [110]:
class DebugMetricCallback(Callback):
    def __init__(self, test, metr):
        super().__init__()
        self.test = test
        self.metric = metr


    def on_epoch_end(self, epoch, logs=None):
        print(self.metric.result())
        x, y = self.test
        y_pred = self.model.predict(x)
        inline_measure = MulticlassF1(num_classes=3)
        inline_measure.update_state(y, y_pred)
        print(self.metric.result().numpy(), inline_measure.result().numpy())
        
        eval_result = self.model.evaluate(x, y, verbose=0)
        print("Evaluation result:", eval_result)


In [112]:
class MulticlassF1(tf.keras.metrics.Metric):

    def __init__(self, name='mc_f1', num_classes=None, **kwargs):
        super(MulticlassF1, self).__init__(name=name, **kwargs)
        self.__zero_support = tf.cast(1e-7, dtype=tf.float16)
        self.__cm = self.add_weight(name='fn', initializer='zeros', shape=[num_classes, num_classes])
        if num_classes is not None:
            self.__num_classes = num_classes

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = K.argmax(y_pred, axis=1)
        y_true = K.argmax(y_true, axis=1)
        m = tf.math.confusion_matrix(y_true, y_pred, num_classes=self.__num_classes, dtype=tf.float32)
        self.__cm.assign_add(m)

    def reset_state(self):
        self.__cm.assign(tf.zeros((self.__num_classes, self.__num_classes)))

    def result(self):
        denominator = 0
        m = self.__cm
        for i in range(m.shape[0]):
            tp = m[i, i]
            fn = K.sum(m[:, i]) - tp
            fp = K.sum(m[i, :]) - tp
            tn = K.sum(K.flatten(m)) - (tp + fn + fp)
            tp = K.cast(tp, dtype=tf.float16)
            tn = K.cast(tn, dtype=tf.float16)
            fp = K.cast(fp, dtype=tf.float16)
            fn = K.cast(fn, dtype=tf.float16)
            precision = tp / ((tp + fp) + self.__zero_support) + self.__zero_support
            recall = tp / (tf.cast(tp + fn, dtype=tf.float16) + self.__zero_support) + self.__zero_support
            denominator += (1 / precision + 1 / recall)
        f1_combined = K.cast(2 * m.shape[0] / denominator, dtype=tf.float32)
        return f1_combined

In [120]:
mcf1 = MulticlassF1(num_classes=3)

resnet50 = get_resnet50_model([32, 32, 3], 3)
resnet50.compile(
    optimizer=Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.metrics.AUC(name='auc'), mcf1],
)

In [ ]:
deb_callback = DebugCallback(resnet50, image_data_generator)
deb_metric_callback = DebugMetricCallback(image_data_generator, mcf1)

resnet50.fit(
    image_data_generator,
    epochs=100,
    batch_size=128,
    # verbose=0,
    shuffle=False,
    callbacks=[deb_metric_callback],
)

Epoch 1/100
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1052/1052 [==============================] - 8s 7ms/step
0.46948242 0.0
Evaluation result: [5.5358500480651855, 0.6588590741157532, 0.0]
263/263 [==============================] - 28s 90ms/step - loss: 1.2546 - auc: 0.7537 - mc_f1: 0.4697
Epoch 2/100
1052/1052 [==============================] - 7s 7ms/step
0.5410156 0.0
Evaluation result: [1.1089696884155273, 0.8319559097290039, 0.0]
263/263 [==============================] - 22s 85ms/s